In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("C:\\Users\\AdNaN\\Desktop\\python\\Projects\\BHP.csv")

: 

In [223]:
df.head()
df.isna().sum()

#stage 2 of the pipeline...
df2=df.drop(columns=['availability','area_type','society','balcony'])
df2['location']=df2['location'].fillna('Whitefield')
df2.isna().sum()


,location,size,total_sqft,bath,price
0,Electronic City Phase II,2 BHK,1056,2.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,95.00
4,Kothanur,2 BHK,1200,2.0,51.00


In [221]:

#stage 3 of the model
df3=df2.copy()
df3['size'].fillna('2 BHK',inplace=True)
df3['bath'].fillna(2.0,inplace=True)

#updating the size column for the given dataframe
df3['size']=df3['size'].apply(lambda x: int(x.split()[0]))
df3.head()

#converting the ranges in the total_sqft column



,location,size,total_sqft,bath,price
0,Electronic City Phase II,2,1056,2.0,39.07
1,Chikka Tirupathi,4,2600,5.0,120.00
2,Uttarahalli,3,1440,2.0,62.00
3,Lingadheeranahalli,3,1521,3.0,95.00
4,Kothanur,2,1200,2.0,51.00


In [217]:
#function to convert range to the average values..
def convertRange(x):
    
    temp=str(x)
    if '-' in temp:
        temp2 = temp.split('-')
        return (float(temp2[0]) + float(temp2[1]))/2
    try : 
        return float(x)
    except:
        return None

In [228]:
df3['total_sqft'] = df3['total_sqft'].apply(convertRange)
df3['total_sqft'].unique()
df3 = df3[df3['size']<20]
df3['size'].unique()

array([ 2,  4,  3,  6,  1,  8,  7,  5, 11,  9, 10, 19, 16, 14, 12, 13, 18],
      dtype=int64)

In [229]:
df3.head(10)


,location,size,total_sqft,bath,price
0,Electronic City Phase II,2,1056.0,2.0,39.07
1,Chikka Tirupathi,4,2600.0,5.0,120.00
2,Uttarahalli,3,1440.0,2.0,62.00
3,Lingadheeranahalli,3,1521.0,3.0,95.00
4,Kothanur,2,1200.0,2.0,51.00
5,Whitefield,2,1170.0,2.0,38.00
6,Old Airport Road,4,2732.0,4.0,204.00
7,Rajaji Nagar,4,3300.0,4.0,600.00
8,Marathahalli,3,1310.0,3.0,63.25
9,Gandhi Bazar,6,1020.0,6.0,370.00


In [233]:
#introducing a new column price per square feet
df3['persqft'] = df3['price']*100000/df3['total_sqft']


#removing those locations which value are less than 10
df3['location']=df3['location'].apply(lambda x:x.strip())
df3.head()

,location,size,total_sqft,bath,price,persqft
0,Electronic City Phase II,2,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861
4,Kothanur,2,1200.0,2.0,51.00,4250.000000


In [236]:
#count locations whose values are less than 10  -> so that we have less values for one hot encoding
locationCount = df3['location'].value_counts() 
lessthan10 = locationCount[locationCount<10]
df3['location']=df3['location'].apply(lambda x: 'other' if x in lessthan10 else x)
df3

C:\Users\AdNaN\AppData\Local\Temp\ipykernel_2276\2580006086.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['location']=df3['location'].apply(lambda x: 'other' if x in lessthan10 else x)


,location,size,total_sqft,bath,price,persqft
0,Electronic City Phase II,2,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861
4,Kothanur,2,1200.0,2.0,51.00,4250.000000
...,...,...,...,...,...,...
13315,Whitefield,5,3453.0,4.0,231.00,6689.834926
13316,other,4,3600.0,5.0,400.00,11111.111111
13317,Raja Rajeshwari Nagar,2,1141.0,2.0,60.00,5258.545136
13318,Padmanabhanagar,4,4689.0,4.0,488.00,10407.336319


In [238]:
#removing outliers by removing the columns which are less than 300
df3 = df3[((df3['total_sqft']/df3['size'])>=300)]
df3.head()

,location,size,total_sqft,bath,price,persqft
0,Electronic City Phase II,2,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861
4,Kothanur,2,1200.0,2.0,51.00,4250.000000


In [239]:
df3.head()


,location,size,total_sqft,bath,price,persqft
0,Electronic City Phase II,2,1056.0,2.0,39.07,3699.810606
1,Chikka Tirupathi,4,2600.0,5.0,120.00,4615.384615
2,Uttarahalli,3,1440.0,2.0,62.00,4305.555556
3,Lingadheeranahalli,3,1521.0,3.0,95.00,6245.890861
4,Kothanur,2,1200.0,2.0,51.00,4250.000000


In [240]:
def remove_outliers_sqft(df):
    df_output = pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.persqft)
        st=np.std(subdf.persqft)
        gen_df=subdf[(subdf.persqft>(m-st)) & (subdf.persqft<=(m+st))]
        df_output=pd.concat([df_output,gen_df],ignore_index=True)
    return df_output  



In [243]:
df3=remove_outliers_sqft(df3)


(4484, 6)

In [244]:
#outlier removal for bhk column 
def bhk_outlier_removal(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        bhk_stats = {}
        for bhk, bhk_df in location_df.groupby('size'):
            bhk_stats[bhk] = {
                'mean': np.mean(bhk_df['persqft']),
                'std': np.std(bhk_df['persqft']),
                'count': bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('size'):
            stats = bhk_stats.get(bhk - 1)
            if stats and stats['count'] > 5:
                exclude_indices = np.append(exclude_indices, bhk_df[bhk_df['persqft'] < (stats['mean'])].index.values)

    return df.drop(exclude_indices, axis='index')

In [289]:

df3.to_csv('Cleaned_Data.csv')
df3=bhk_outlier_removal(df3)
df3.drop(columns=['persqft'])


,location,size,total_sqft,bath,price
0,1st Block Jayanagar,3,1630.0,3.0,194.0
1,1st Block Jayanagar,2,1235.0,2.0,148.0
2,1st Block Koramangala,2,1415.0,2.0,110.0
3,1st Block Koramangala,3,2350.0,3.0,210.0
4,1st Phase JP Nagar,4,2825.0,4.0,250.0
...,...,...,...,...,...
4472,other,2,1012.0,2.0,62.0
4477,other,2,1155.0,2.0,64.0
4479,other,2,1200.0,2.0,70.0
4480,other,2,1095.0,2.0,57.0


In [251]:
X=df3.drop(columns=['price'])
y=df3['price']

In [258]:
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [283]:
column_trans = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore',sparse=False), ['location']),
    remainder='passthrough'
)
scaler=StandardScaler()
lr=LinearRegression()
pipe = make_pipeline(
    column_trans,
    LinearRegression()  # Linear regression with normalization
)

In [285]:
pipe.fit(X_train,y_train)

C:\Users\AdNaN\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['location'])])),
                ('linearregression', LinearRegression())])

In [288]:
y_pred_lr=pipe.predict(X_test)
pipe.score(X_test,y_test)

0.9765535968677611

In [290]:
import pickle
pickle.dump(pipe,open('Model.pkl','wb'))